In [1]:
'''
https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
https://towardsdatascience.com/machine-learning-word-embedding-sentiment-classification-using-keras-b83c28087456
http://ruder.io/deep-learning-nlp-best-practices/
https://skymind.ai/wiki/word2vec
https://medium.com/@ppasumarthi_69210/word-embeddings-in-keras-be6bb3092831
https://towardsdatascience.com/neural-network-embeddings-explained-4d028e6f0526
https://medium.com/@dcameronsteinke/tf-idf-vs-word-embedding-a-comparison-and-code-tutorial-5ba341379ab0
https://medium.com/@b.terryjack/nlp-everything-about-word-embeddings-9ea21f51ccfe
https://towardsdatascience.com/word-embeddings-exploration-explanation-and-exploitation-with-code-in-python-5dac99d5d795
'''

'\nhttps://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html\nhttps://towardsdatascience.com/machine-learning-word-embedding-sentiment-classification-using-keras-b83c28087456\nhttp://ruder.io/deep-learning-nlp-best-practices/\nhttps://skymind.ai/wiki/word2vec\nhttps://medium.com/@ppasumarthi_69210/word-embeddings-in-keras-be6bb3092831\nhttps://towardsdatascience.com/neural-network-embeddings-explained-4d028e6f0526\nhttps://medium.com/@dcameronsteinke/tf-idf-vs-word-embedding-a-comparison-and-code-tutorial-5ba341379ab0\nhttps://medium.com/@b.terryjack/nlp-everything-about-word-embeddings-9ea21f51ccfe\nhttps://towardsdatascience.com/word-embeddings-exploration-explanation-and-exploitation-with-code-in-python-5dac99d5d795\n'

In [2]:
from __future__ import print_function

import csv
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM, Dropout
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('rslp')

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/konpyro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/konpyro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/konpyro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package rslp to /home/konpyro/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [3]:
# X: input, Y:output
X = []
Y = []
with open('MoodyLyricsExtended4Q.csv') as file:
    reader = csv.reader(file, delimiter = ',')
    count = 0
    for row in reader:
        if count > 0:
            X.append(row[4])
            Y.append(row[3])
        count = count + 1

In [4]:
#Y = np.concatenate(y_train, y_test)
y_t = np.zeros((len(Y), 4))
for i  in range(len(Y)):
    if Y[i] == 'happy':
        y_t[i][:] = [1, 0, 0, 0]
    elif Y[i] == 'angry':
        y_t[i][:] = [0, 1, 0, 0]
    elif Y[i] == 'sad':
        y_t[i][:] = [0, 0, 1, 0]
    elif Y[i] == 'relaxed':
        y_t[i][:] = [0, 0, 0, 1]
Y = y_t

In [5]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [6]:
t = Tokenizer()
total_lyrics = x_train + x_test
t.fit_on_texts(total_lyrics)

max_length = max([len(s.split()) for s in total_lyrics])
vocab_size = len(t.word_index) + 1
EMBEDDING_DIM = 100

x_train_tokens = t.texts_to_sequences(x_train)
x_test_tokens = t.texts_to_sequences(x_test)

x_train_pad = pad_sequences(x_train_tokens, padding='post')
x_test_pad = pad_sequences(x_test_tokens, padding='post')

In [7]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

lyric_lines = list()
lines = X

for line in lines:
    tokens = word_tokenize(line)

    tokens = [w.lower() for w in tokens]

    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]

    words = [word for word in stripped if word.isalpha()]

    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    lyric_lines.append(words)

In [8]:
t = Tokenizer()
t.fit_on_texts(lyric_lines)
sequences = t.texts_to_sequences(lyric_lines)

word_index = t.word_index
print('Found %s unique tokens.' %len(word_index))

review_pad = pad_sequences(sequences, maxlen=max_length)
sentiment = np.asarray(Y)
print('Shape of lyric tensor:', review_pad.shape)
print('Shape of sentiment tensor;', sentiment.shape)

Found 16375 unique tokens.
Shape of lyric tensor: (18115, 408)
Shape of sentiment tensor; (18115, 4)


In [9]:
import os

embeddings_index = {}
f = open(os.path.join('', 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [10]:
word_index = t.word_index
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        

In [11]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)


In [12]:
''''
filename = 'lyric_embedding_word2vec'
model.wv.save(filename)

import os

embeddings_index = {}
f = open(os.path.join('', 'lyric_embedding_word2vec.txt'), encoding='latin-1')
for line in f:
  print(line)
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:])
  embeddings_index[word] = coefs
f.close()
'''

"'\nfilename = 'lyric_embedding_word2vec'\nmodel.wv.save(filename)\n\nimport os\n\nembeddings_index = {}\nf = open(os.path.join('', 'lyric_embedding_word2vec.txt'), encoding='latin-1')\nfor line in f:\n  print(line)\n  values = line.split()\n  word = values[0]\n  coefs = np.asarray(values[1:])\n  embeddings_index[word] = coefs\nf.close()\n"

In [13]:
from keras.initializers import Constant

print('Biuld model...')
model = Sequential()

model.add(embedding_layer)
#model.add(LSTM(32, dropout=0.2, return_sequences=True, recurrent_dropout=0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Biuld model...


In [14]:
VALIDATION_SPLIT = 0.33

indices = np.arange(review_pad.shape[0])
np.random.shuffle(indices)
review_pad = review_pad[indices]
sentiment = sentiment[indices]

x_train_pad, x_test_pad, y_train, y_test = train_test_split(review_pad, sentiment, test_size=0.33)

In [15]:
print('Shape of x_train_pad tensor:', x_train_pad.shape)
print('Shape of y_train tensor:', y_train.shape)
print('Shape of x_test_pad tensor:', x_test_pad.shape)
print('Shape of y_test tensor:', y_test.shape)

Shape of x_train_pad tensor: (12137, 408)
Shape of y_train tensor: (12137, 4)
Shape of x_test_pad tensor: (5978, 408)
Shape of y_test tensor: (5978, 4)


In [ ]:
print('Train model...')

model.fit(x_train_pad, y_train, epochs=20, validation_data=(x_test_pad, y_test), verbose=2)

Train model...


In [ ]:
loss, acc = model.evaluate(x_test_pad, y_test)
print('Test loss:', loss)
print('Test accuracy:', acc)

LSTM(32),
split: 0.33,
epochs: 5,
dropout=r_dropout = 0.2,
-> **loss: 1.2, acc: 0.46**

LSTM(64),
split: 0.33,
epochs: 5,
dropout=r_dropout = 0.2,
-> **loss: 1.18, acc: 0.49**

LSTM(64),
split: 0.33,
epochs: 10,
dropout=r_dropout = 0.2,
loss_fun = kullback_leibler_divergence
-> **loss: 1.19, acc: 0.48**

LSTM(64),
split: 0.33,
epochs: 5,
dropout=r_dropout = 0.1,
-> **loss: 1.2, acc: 0.45**

LSTM(64),
split: 0.33,
epochs: 25,
dropout=r_dropout = 0.2,
-> **loss: 1.22, acc: 0.48**

LSTM(128),
split: 0.33,
epochs: 5,
dropout=r_dropout = 0.2,
-> **loss: 1.2, acc: 0.41**

LSTM(32),
LSTM(32),
split: 0.33,
epochs: 5,
dropout=r_dropout = 0.2,
-> **loss: 1.18, acc: 0.45**

LSTM(64),
LSTM(64),
split: 0.33,
epochs: 5,
dropout=r_dropout = 0.2,
-> **loss: 1.24, acc: 0.42**

